# Import Library

In [1]:
import pandas as pd
import json

# Load Data

In [2]:
info = pd.read_csv('Data/superhero_info - superhero_info.csv')
info.head()

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}"
2,Abin Sur|DC Comics,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}"
3,Abomination|Marvel Comics,Male,Human / Radiation,bad,No Hair,green,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
4,Absorbing Man|Marvel Comics,Male,Human,bad,No Hair,blue,Unknown,"{'Height': '193.0 cm', 'Weight': '122.0 kg'}"


In [3]:
powers = pd.read_csv('Data/superhero_powers - superhero_powers.csv')
powers.head()

,hero_names,Powers
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed"
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super..."
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du..."
3,Abin Sur,Lantern Power Ring
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt..."


# Inspect Data before Cleaning

In [4]:
info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Hero|Publisher  463 non-null    object
 1   Gender          463 non-null    object
 2   Race            463 non-null    object
 3   Alignment       463 non-null    object
 4   Hair color      463 non-null    object
 5   Eye color       463 non-null    object
 6   Skin color      463 non-null    object
 7   Measurements    463 non-null    object
dtypes: object(8)
memory usage: 29.1+ KB


In [5]:
powers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 667 entries, 0 to 666
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   hero_names  667 non-null    object
 1   Powers      667 non-null    object
dtypes: object(2)
memory usage: 10.5+ KB


# Clean Data

I. Clean the files and combine them into one final DataFrame.

This dataframe should have the following columns:
- Hero (Just the name of the Hero)
- Publisher
- Gender
- Eye color
- Race
- Hair color
- Height (numeric)
- Skin color
- Alignment
- Weight (numeric)
- Plus, one-hot-encoded columns for every power that appears in the dataset. E.g.:
    - Agility
    - Flight
    - Superspeed
    - etc.
    
Hint: There is a space in "100 kg" or "52.5 cm"

## 1. 'Hero' column should just contain the name of the Hero

In [6]:
# Check the current values under the Hero|Publisher column
info['Hero|Publisher'].value_counts()

Spider-Man|Marvel Comics       3
Black Canary|DC Comics         2
Nova|Marvel Comics             2
Toxin|Marvel Comics            2
Batman|DC Comics               2
                              ..
Fabian Cortez|Marvel Comics    1
Exodus|Marvel Comics           1
Evilhawk|Marvel Comics         1
Evil Deadpool|Marvel Comics    1
Zoom|DC Comics                 1
Name: Hero|Publisher, Length: 457, dtype: int64

In [7]:
## Separate the 'Hero|Publisher' column into 2 columns and save the 2 new columns into the dataframe
info[['Hero','Publisher']] = info['Hero|Publisher'].str.split('|',expand=True)
info.head(2)

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics


In [8]:
## drop the original column
info = info.drop(columns=['Hero|Publisher'])
info.head(2)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher
0,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics
1,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics


## 2. Separate the "Measurements" column into "Height" (Numerical) and "Weight" (Numerical)

### 2.1  Convert a string column of dictionaries into actual dictionaries.

In [9]:
## use .str.replace to replace all single quotes
info['Measurements'] = info['Measurements'].str.replace("'",'"')
## Apply the json.loads to the full column
info['Measurements'] = info['Measurements'].apply(json.loads)
info['Measurements'].head(2)

0    {'Height': '203.0 cm', 'Weight': '441.0 kg'}
1     {'Height': '191.0 cm', 'Weight': '65.0 kg'}
Name: Measurements, dtype: object

### 2.2 Unpack a column of dictionaries into separate columns

In [10]:
measurements = info['Measurements'].apply(pd.Series)
measurements

,Height,Weight
0,203.0 cm,441.0 kg
1,191.0 cm,65.0 kg
2,185.0 cm,90.0 kg
3,203.0 cm,441.0 kg
4,193.0 cm,122.0 kg
...,...,...
458,183.0 cm,83.0 kg
459,165.0 cm,52.0 kg
460,66.0 cm,17.0 kg
461,170.0 cm,57.0 kg


### 2.3 Concatenate these 2 new columns to original dataframe and drop the original column.

In [11]:
# concat measurements with original dataframe
info = pd.concat([info,measurements], axis=1)
info.head(2)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher,Height,Weight
0,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics,203.0 cm,441.0 kg
1,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics,191.0 cm,65.0 kg


In [12]:
# drop the original column
info = info.drop(columns=['Measurements'])

In [13]:
# check to confirm that the measurements column has dropped
info.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0 cm,441.0 kg
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0 cm,65.0 kg
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0 cm,90.0 kg
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,203.0 cm,441.0 kg
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,193.0 cm,122.0 kg


### 2.4 Replace multiple characters at once within a string column

203.0 cm -> 203.0


Remove ' cm' from the Height column

In [14]:
# Make a list of all characters to replace
to_replace = [" cm"]
# run a loop to replace all of the characters in the list at once
for char in to_replace:
    info['Height'] = info['Height'].str.replace(char,"",regex=False)
    
info['Height'].head()

0    203.0
1    191.0
2    185.0
3    203.0
4    193.0
Name: Height, dtype: object

Remove ' kg' from the Weight column

In [15]:
# Make a list of all characters to replace
to_replace = [" kg"]
# run a loop to replace all of the characters in the list at once
for char in to_replace:
    info['Weight'] = info['Weight'].str.replace(char,"",regex=False)
    
info['Weight'].head()

0    441.0
1     65.0
2     90.0
3    441.0
4    122.0
Name: Weight, dtype: object

### 2.5 Convert dtype of values in "Height" and "Weight" columns to numerical

In [16]:
info['Height'].astype(float)

0      203.0
1      191.0
2      185.0
3      203.0
4      193.0
       ...  
458    183.0
459    165.0
460     66.0
461    170.0
462    185.0
Name: Height, Length: 463, dtype: float64

In [17]:
info['Weight'].astype(float)

0      441.0
1       65.0
2       90.0
3      441.0
4      122.0
       ...  
458     83.0
459     52.0
460     17.0
461     57.0
462     81.0
Name: Weight, Length: 463, dtype: float64

In [18]:
# check
info.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0,65.0
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0,90.0
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,203.0,441.0
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,193.0,122.0


In [19]:
#check dtype
info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Gender      463 non-null    object
 1   Race        463 non-null    object
 2   Alignment   463 non-null    object
 3   Hair color  463 non-null    object
 4   Eye color   463 non-null    object
 5   Skin color  463 non-null    object
 6   Hero        463 non-null    object
 7   Publisher   463 non-null    object
 8   Height      463 non-null    object
 9   Weight      463 non-null    object
dtypes: object(10)
memory usage: 36.3+ KB


In [20]:
info['Height'].mean()

TypeError: Could not convert 203.0191.0185.0203.0193.0185.0178.0191.0188.0193.0180.0244.0257.0188.0183.0165.0163.0183.0180.0211.0183.0229.0213.0178.0185.0175.0183.0173.0193.0185.0165.0163.0183.0198.0178.0168.0183.0180.0183.0203.0183.0165.0170.0165.0168.0188.0178.0178.0198.0175.0180.0173.0201.0188.0165.0180.0198.0191.0191.0188.0165.0170.0178.0183.0185.0170.0188.0183.0170.0191.0185.0188.0188.0168.0165.0175.0178.0218.0183.0165.0196.0198.0170.0183.0157.0183.0170.0203.0175.0183.0188.0193.0198.0180.0193.0175.0185.0173.0175.0201.0175.0180.0163.0170.0175.0185.0183.0226.0226.0183.0183.0180.0168.0198.0191.0165.0183.0185.0267.0168.0198.0122.0173.0183.0188.0185.0193.0193.0185.0188.0193.0198.0201.0201.0188.0175.0188.0173.0175.0244.0196.0193.0168.0180.0175.0185.0178.0168.0193.0188.0191.0183.0196.0188.0175.0975.0165.0193.0173.0188.0180.0183.0183.0157.0183.0142.0211.0180.0876.0185.0183.0185.0188.062.5198.0168.0175.0183.0198.0178.0178.0188.0180.0178.0183.0178.0701.0188.0188.0183.0170.0183.0185.0191.016